# Создание представлений данных

### 1. Подготовка

In [1]:

import time
import mysql.connector as mysql


### 2. Создание функции для БД

In [ ]:

if __name__ == '__main__':
    def alter_incert_db(
        sql_query,
        host_ = '', 
        user_ = '', 
        password_ = '',
        database_ = 'tender_project'
    ):
        '''Функция для создания объектов в БД имеет следующие параметры:
            sql_query - sql запрос;
            host- хост БД (данные по-умолчанию);
            user - логин БД (данные по-умолчанию);
            password - пароль (данные по-умолчанию);
            database - имя (данные по-умолчанию).'''
        try:
            with mysql.connect(
                host = host_,
                user = user_,
                password = password_,
                database = database_
            ) as connection_pet:
                with connection_pet.cursor() as cursor_pet:
                    cursor_pet.execute(sql_query)
                    connection_pet.commit()
                    print(connection_pet)
        except mysql.Error as e:
            print(e)


### 3. SQL запросы для создания представлений данных

In [3]:

'''   Блок написания переменных со скриптами SQL   '''

create_view_dict_tenders = """
    CREATE OR REPLACE VIEW v_dict_tenders AS
    WITH join_tru_marks AS(
    	SELECT 
    		DISTINCT mu.`_id`, 
    		FIRST_VALUE(dm.`_id`) OVER (PARTITION BY mu.`_id` ORDER BY dm.`true_name` ASC) AS marks_id
    	FROM marks_users mu JOIN dict_marks dm 
    	ON mu.marks = dm.`_id`
    ),
    join_tru_users AS(
    	SELECT 
    		DISTINCT mu.`_id`,
    		FIRST_VALUE(du.`_id`) OVER (PARTITION BY mu.`_id` ORDER BY du.`_id` ASC) AS users_id
    	FROM marks_users mu JOIN dict_users du 
    	ON mu.users = du.`_id`
    ),
    return_ AS(
    	SELECT dt.*,  mk.marks_id, us.users_id
    	FROM dict_tenders dt 
    	JOIN join_tru_marks mk ON mk.`_id` = dt.`_id`
    	JOIN join_tru_users us ON us.`_id` = dt.`_id`  
    )
    SELECT * FROM return_;
"""
create_view_dict_marks = """
    CREATE OR REPLACE VIEW v_dict_marks AS
    SELECT * FROM dict_marks dm;
"""

create_view_dict_users = """
    CREATE OR REPLACE VIEW v_dict_users AS
    SELECT * FROM dict_users du;
"""

create_view_dict_region = """
    CREATE OR REPLACE VIEW v_dict_region AS
    SELECT * FROM dict_region du;
"""

create_view_dict_status = """
    CREATE OR REPLACE VIEW v_dict_status AS
    SELECT * FROM dict_status du;
"""

create_view_dict_placingways = """
    CREATE OR REPLACE VIEW v_dict_placingways AS
    SELECT * FROM dict_placingways du;
"""

create_view_participants = """
    CREATE OR REPLACE VIEW v_participants AS
    SELECT ps.* FROM participants ps
    LEFT JOIN v_dict_tenders v
    ON ps.`_id` = v.`_id` 
    WHERE NOT v.`_id` IS NULL;
"""

create_view_products = """
    CREATE OR REPLACE VIEW v_products AS
    SELECT ps.* FROM products ps
    LEFT JOIN v_dict_tenders v
    ON ps.`_id` = v.`_id` 
    WHERE NOT v.`_id` IS NULL;
"""


### 4. Переменная со списком sql запросов

In [4]:

lst_sql = [
    create_view_dict_tenders,
    create_view_dict_marks,
    create_view_dict_users,
    create_view_dict_region,
    create_view_dict_status,
    create_view_dict_placingways,
    create_view_participants,
    create_view_products
]


### 5. Цикл создания представлений данных (транзакции на каждый запрос)

In [5]:

for x in lst_sql:
    alter_incert_db(x)
    time.sleep(1)
